In [ ]:
import pandas as pd
try:
    import numpy as np
except ImportError as e:
    print(f"Numpy is required for this feature: {e}")
    raise
import matplotlib.pyplot as plt
def load_data(file_path):
    try:
        data = pd.read_csv(file_path)
        return data
    except Exception as e:
        print(f"Error loading data: {e}")
        return None
def plot_temperature(data):
    plt.figure(figsize=(10, 5))
    plt.plot(data['Date'], data['MinTemp'], label='Min Temperature')
    plt.plot(data['Date'], data['MaxTemp'], label='Max Temperature')
    plt.xlabel('Date')
    plt.ylabel('Temperature')
    plt.title('Min and Max Temperature Over Time')
    plt.legend()
    plt.show()
# Example usage
# data = load_data('path_to_your_file.csv')
# if data is not None:
#     plot_temperature(data)
params = {
    "start_date": "2023-11-22",
    "end_date": "2024-11-22"
}
